<a href="https://colab.research.google.com/github/redwon007/JobShopPSO/blob/main/15x15_jobShop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import os
from copy import deepcopy
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
from datetime import datetime
import math
from random import randint, random

In [29]:
nbMachines = input("nbMachines?")
nbJobs = input("nbJobs?")
nbDataSet = int(input("nbDataSet?"))

path = "tai" + nbMachines + "_" + nbJobs + ".txt"
fichier = open(path, "r")

for i in range(34*(nbDataSet-1) + 3 ):
    line = fichier.readline()
    
times = []
for i in range(20):
    line = [int(i) for i in fichier.readline().strip().split()]
    times.append(line)
    
line = fichier.readline()#read line "Machines\n"

machines = []
for i in range(20):
    line = [int(i) for i in fichier.readline().strip().split()]
    machines.append(line)
    
fichier.close()

nbMachines?20
nbJobs?15
nbDataSet?1


In [30]:
print(times)
print(machines)

[[25, 75, 75, 76, 38, 62, 38, 59, 14, 13, 46, 31, 57, 92, 3], [67, 5, 11, 11, 40, 34, 77, 42, 35, 96, 22, 55, 21, 29, 16], [22, 98, 8, 35, 59, 31, 13, 46, 52, 22, 18, 19, 64, 29, 70], [99, 42, 2, 35, 11, 92, 88, 97, 21, 56, 17, 43, 27, 19, 23], [50, 5, 59, 71, 47, 39, 82, 35, 12, 2, 39, 42, 52, 65, 35], [48, 57, 5, 2, 60, 64, 86, 3, 51, 26, 34, 39, 45, 63, 54], [40, 43, 50, 71, 46, 99, 67, 34, 6, 95, 67, 54, 29, 30, 60], [59, 3, 85, 6, 46, 49, 5, 82, 18, 71, 48, 79, 62, 65, 76], [65, 55, 81, 15, 32, 52, 97, 69, 82, 89, 69, 87, 22, 71, 63], [70, 74, 52, 94, 14, 81, 24, 14, 32, 39, 67, 59, 18, 77, 50], [18, 6, 96, 53, 35, 99, 39, 18, 14, 90, 64, 81, 89, 48, 80], [44, 75, 12, 13, 74, 59, 71, 75, 30, 93, 26, 30, 84, 91, 93], [39, 56, 13, 29, 55, 69, 26, 7, 55, 48, 22, 46, 50, 96, 17], [57, 14, 8, 13, 95, 53, 78, 24, 92, 90, 68, 87, 43, 75, 94], [93, 92, 18, 28, 27, 40, 56, 83, 51, 15, 97, 48, 53, 78, 39], [47, 34, 42, 28, 11, 11, 30, 14, 10, 4, 20, 92, 19, 59, 28], [69, 82, 64, 40, 27, 82,

In [42]:
def vectorSolToMatrixSol(vectorSol):
    matrixSol = [[] for i in range(len(machines[0]))]
    operation = [1] * len(machines)
    for job in vectorSol:
        job = int(job)
        matrixSol[machines[job-1][operation[job-1]-1]-1].append((job,operation[job-1]))
        operation[job-1] += 1
    return(matrixSol)


In [64]:
def createGantt(solution):
    gantt = deepcopy(solution)
    nbJobs = len(machines)
    toPlaceList = [1] * nbJobs
    endTimeList = [0] * nbJobs
    i,j = 0,0
    while(i<len(machines[0])):
    #for i in range(len(machines)):
        
        while(j<nbJobs):
        #for j in range(nbJobs):
            #print(i,j, " ", toPlaceList , " " , endTimeList)
            refresh = False
            for job, toPlace in enumerate(toPlaceList):
                
                if(solution[i][j][0] == job+1 and solution[i][j][1] == toPlace):
                    if(j==0 or gantt[i][j-1] != solution[i][j-1]):
                        #print(toPlaceList , " " , endTimeList)      
                        #print(i,j)
                        refresh = True
                        if(j == 0):
                            previous = 0
                        else:
                            previous = gantt[i][j-1][1]
                        startTime = max(previous, endTimeList[job])
                        endTime = startTime + times[job][solution[i][j][1]-1]
                        gantt[i][j] = (startTime, endTime)
                        toPlaceList[job] += 1
                        endTimeList[job] = endTime
                        #j = 0
            if(refresh == True):
                break
            j += 1
        i += 1
        j = 0    
        if(refresh == True):
            i=0
    return(gantt)
    

In [33]:
def makespan(vectorSolution):
    gantt = createGantt(vectorSolToMatrixSol(vectorSolution))
    makespan = max(gantt[0][-1])
    for lastOp in gantt:     
      lastRelease = max(lastOp[-1])
      if(lastRelease > makespan):
        makespan = lastRelease
    return(makespan)

In [34]:
def computeNextVelocity(velocity, pos, bPos, bbPos):
    nextVelocity = velocity + (bPos - pos) + (bbPos - pos)
    return nextVelocity

In [35]:
def arrangeSol(sol, continuousSol):
    newSol = np.array([])
    while(np.size(sol) != 0):
        minIndex = np.where(continuousSol == np.amin(continuousSol))[0]
        
        for i in minIndex:
            newSol = np.append(newSol, sol[i])
        inds_to_delete = sorted(minIndex, reverse=True)
        for i in inds_to_delete:
            sol = np.delete(sol, i)
            continuousSol = np.delete(continuousSol, i)
    return(newSol)

In [36]:
def firstNotSchedule(newPosition, position):
  scheduled = [0] * len(machines)
  for i, job in enumerate(position):
    if(np.count_nonzero(newPosition == job) == scheduled[int(job)-1]):
      return(job)
    else:
      scheduled[int(job)-1] +=1

In [37]:
def newParticle(particle, pBestParticle, gBestParticle):
  newPosition = np.array([])
  #pIndex, pBIndex, gBIndex = 0, 0, 0
  for j in range(len(particle)):
    i = 0
    nbJobs = len(times)
    wc, wb, wg = 0.2, 0.3, 0.5
    u = random()
    if(u <= wc):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, particle))
    elif(u <= wb + wg):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, pBestParticle))
    elif(wc+wb < u):
      newPosition = np.append(newPosition, firstNotSchedule(newPosition, gBestParticle))

  return(newPosition)


In [38]:
def createSol(nbJobs, nbMachines):
  sol = np.array([])
  for i in range(nbMachines):
    for j in range(1, nbJobs + 1):
      sol = np.append(sol, j)
  return(sol)

In [55]:
sol = createSol(20, 15)
print(sol)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.
 15. 16. 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.
 13. 14. 15. 16. 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.
 11. 12. 13. 14. 15. 16. 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.
  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.
  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20.  1.  2.  3.  4.
  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20.  1.  2.
  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20.
  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.
 15. 16. 17. 18. 19. 20.  1.  2.  3.  4.  5.  6.  7

In [47]:
print(vectorSolToMatrixSol(sol))

[[(13, 1), (2, 2), (8, 2), (3, 4), (11, 4), (17, 5), (6, 6), (7, 6), (10, 6), (14, 6), (15, 8), (1, 9), (12, 10), (5, 11), (20, 11), (9, 12), (4, 13), (19, 13), (16, 14), (18, 14)], [(8, 1), (10, 1), (4, 3), (5, 3), (6, 3), (7, 3), (11, 3), (1, 4), (14, 5), (19, 5), (2, 6), (15, 7), (17, 7), (9, 8), (20, 8), (12, 9), (13, 10), (16, 12), (3, 14), (18, 15)], [(3, 1), (7, 1), (12, 1), (19, 2), (8, 3), (18, 3), (5, 4), (1, 6), (16, 6), (9, 7), (4, 9), (14, 10), (6, 11), (11, 11), (15, 11), (2, 12), (13, 12), (20, 14), (10, 15), (17, 15)], [(1, 1), (6, 1), (11, 1), (15, 1), (17, 1), (19, 1), (3, 2), (14, 2), (2, 3), (10, 3), (4, 5), (12, 5), (8, 8), (13, 8), (18, 8), (9, 10), (16, 11), (7, 12), (20, 12), (5, 13)], [(9, 1), (10, 2), (8, 4), (14, 4), (2, 5), (4, 6), (1, 7), (16, 7), (3, 8), (5, 8), (12, 8), (11, 9), (19, 9), (20, 9), (7, 11), (13, 11), (17, 11), (18, 11), (15, 12), (6, 15)], [(2, 1), (16, 1), (9, 2), (20, 3), (6, 4), (15, 4), (12, 6), (3, 7), (19, 8), (14, 9), (17, 9), (4, 10

In [56]:
def jobShopPSO():
    swarm = [{"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParti cle" : np.array([]) , "velocity" : np.array([])},
             {"position" : np.random.permutation(sol) , "pBestParticle" : np.array([]) , "cMaxBestParticle" : np.array([]) , "velocity" : np.array([])},

        ]
    gBestParticle = swarm[0]["position"]
    gBestParticleCmax = makespan(gBestParticle)
    for i, particle in enumerate(swarm):
        particle["pBestParticle"] = particle["position"]
        particle["pBestParticleCmax"] = makespan(particle["pBestParticle"])
        particle["velocity"] = 0
        if(particle["pBestParticleCmax"] < gBestParticleCmax):
            gBestParticle = particle["position"]
            gBestParticleCmax = particle["pBestParticleCmax"]
        
    iteration = 0
    while(iteration < 10):
        iteration +=1
        print('\n\niteration', iteration)
        for i, particle in enumerate(swarm):
            #particle["velocity"] = computeNextVelocity(particle["velocity"], particle["position"], particle["pBestParticle"], gBestParticle)
            particle["position"] = newParticle(particle["position"], particle["pBestParticle"], gBestParticle)
            pCmax = makespan(particle["position"])
            if(pCmax < particle["pBestParticleCmax"]):
                particle["pBestParticle"] = particle["position"]
                particle["pBestParticleCmax"] = pCmax
                if(pCmax < gBestParticleCmax):
                    gBestParticle = particle["position"]
                    gBestParticleCmax = pCmax
            print("particle",i," cmax = ", pCmax)
    return((gBestParticle, gBestParticleCmax))

In [65]:
a = jobShopPSO()



iteration 1
particle 0  cmax =  3129
particle 1  cmax =  2658
particle 2  cmax =  2566
particle 3  cmax =  2492
particle 4  cmax =  2880


iteration 2
particle 0  cmax =  3139
particle 1  cmax =  2827
particle 2  cmax =  2529
particle 3  cmax =  2492
particle 4  cmax =  2905


iteration 3
particle 0  cmax =  3132
particle 1  cmax =  2779
particle 2  cmax =  2700
particle 3  cmax =  2492
particle 4  cmax =  2937


iteration 4
particle 0  cmax =  2953
particle 1  cmax =  2429
particle 2  cmax =  2674
particle 3  cmax =  2534
particle 4  cmax =  2786


iteration 5
particle 0  cmax =  2907
particle 1  cmax =  2429
particle 2  cmax =  2765
particle 3  cmax =  2315
particle 4  cmax =  2616


iteration 6
particle 0  cmax =  2994
particle 1  cmax =  2462
particle 2  cmax =  3066
particle 3  cmax =  2315
particle 4  cmax =  2811


iteration 7
particle 0  cmax =  2990
particle 1  cmax =  2373
particle 2  cmax =  3007
particle 3  cmax =  2315
particle 4  cmax =  2638


iteration 8
particle 0  c

In [23]:
print(a)

(array([13., 16., 11., 19., 12., 11.,  3.,  2.,  3.,  9., 20.,  9.,  6.,
        2.,  1.,  7., 10., 15., 20.,  7., 11., 18., 17.,  1., 18.,  7.,
        9.,  6., 12., 19.,  9.,  3.,  8.,  2.,  4.,  2., 10., 10.,  8.,
        4., 20., 17., 19., 12., 14., 15.,  2.,  5.,  6.,  1., 15.,  2.,
       14.,  2., 20.,  1.,  4.,  2., 13., 13.,  1., 12., 17.,  8., 20.,
       16.,  4.,  9.,  7.,  5., 16., 18., 14., 19.,  3., 14.,  1.,  3.,
        3.,  1., 10., 13.,  8.,  2., 13., 16.,  4., 19., 17.,  7., 15.,
        5.,  6.,  2.,  6., 19., 14.,  9.,  9.,  4., 11., 13., 11.,  7.,
        4., 13., 16., 18., 17.,  2.,  4., 18.,  1., 14.,  7.,  5., 20.,
       19., 18.,  2.,  6.,  9., 20.,  8.,  5.,  7., 18., 10.,  1., 14.,
       17.,  4., 12.,  3., 15.,  3.,  1., 12.,  6.,  6.,  5.,  8., 15.,
        1.,  9.,  9.,  8.,  4., 12.,  6., 17., 16.,  9., 15.,  5.,  5.,
       15., 13.,  3.,  7., 20.,  8.,  4., 14., 19.,  7., 17., 17., 10.,
       20., 18.,  9., 12., 19., 15.,  3.,  9., 18.,  5., 15.,  

In [17]:
print(vectorSolToMatrixSol(a[0]))
print(a[1])
createGantt(vectorSolToMatrixSol(a[0]))

[[(12, 3), (11, 4), (8, 3), (9, 5), (10, 6), (4, 6), (13, 8), (2, 11), (3, 9), (15, 10), (6, 12), (7, 13), (5, 14), (1, 14), (14, 15)], [(3, 1), (15, 6), (9, 6), (8, 8), (5, 11), (7, 7), (13, 11), (14, 8), (12, 12), (11, 12), (6, 10), (2, 14), (10, 11), (4, 14), (1, 15)], [(12, 1), (4, 2), (10, 4), (13, 4), (1, 6), (5, 7), (9, 7), (3, 10), (8, 9), (6, 9), (7, 11), (2, 15), (15, 13), (14, 14), (11, 15)], [(7, 2), (1, 5), (6, 2), (13, 5), (10, 8), (2, 12), (11, 11), (12, 13), (8, 12), (15, 12), (9, 15), (3, 14), (5, 15), (14, 13), (4, 15)], [(11, 1), (2, 1), (1, 3), (15, 5), (14, 3), (5, 5), (6, 6), (13, 10), (9, 10), (7, 9), (4, 8), (8, 11), (3, 13), (12, 15), (10, 14)], [(13, 1), (4, 1), (6, 1), (2, 2), (8, 2), (11, 5), (3, 8), (12, 8), (14, 5), (9, 8), (5, 10), (7, 10), (1, 13), (10, 15), (15, 15)], [(10, 1), (1, 1), (15, 4), (3, 5), (5, 3), (9, 3), (12, 5), (4, 4), (11, 7), (2, 9), (13, 6), (14, 6), (7, 6), (8, 14), (6, 14)], [(5, 1), (11, 2), (2, 3), (1, 4), (8, 4), (7, 3), (12, 7),

[[(90, 165),
  (165, 173),
  (250, 259),
  (647, 660),
  (660, 677),
  (690, 784),
  (883, 960),
  (960, 978),
  (978, 1032),
  (1111, 1119),
  (1438, 1525),
  (1525, 1562),
  (1566, 1608),
  (1625, 1695),
  (1727, 1762)],
 [(0, 4),
  (374, 400),
  (660, 747),
  (747, 809),
  (1000, 1093),
  (1101, 1120),
  (1120, 1168),
  (1168, 1224),
  (1224, 1269),
  (1269, 1362),
  (1362, 1388),
  (1388, 1487),
  (1487, 1556),
  (1703, 1795),
  (1795, 1878)],
 [(0, 47),
  (138, 161),
  (253, 313),
  (342, 362),
  (365, 380),
  (585, 680),
  (747, 829),
  (1032, 1100),
  (1100, 1106),
  (1106, 1184),
  (1332, 1375),
  (1487, 1520),
  (1526, 1609),
  (1720, 1727),
  (1727, 1800)],
 [(183, 258),
  (339, 365),
  (365, 426),
  (426, 457),
  (743, 832),
  (978, 1029),
  (1029, 1119),
  (1269, 1336),
  (1359, 1439),
  (1439, 1526),
  (1526, 1541),
  (1541, 1576),
  (1608, 1704),
  (1704, 1720),
  (1795, 1804)],
 [(0, 90),
  (90, 164),
  (165, 175),
  (337, 374),
  (374, 425),
  (520, 556),
  (654, 685),


In [ ]:

df = pd.DataFrame([
    dict(Task="Job A", Start=convert_to_datetime(0), Finish=convert_to_datetime(3), Resource="Alex"),
    dict(Task="Job B", Start=convert_to_datetime(4), Finish=convert_to_datetime(5), Resource="Alex"),
    dict(Task="Job C", Start=convert_to_datetime(0), Finish=convert_to_datetime(2), Resource="Max")
])
#Maurice clerc PSO

NameError: name 'convert_to_datetime' is not defined

In [ ]:
def convert_to_datetime(x):
  return datetime.fromtimestamp(31536000+x*24*3600).strftime("%Y-%d-%m")

In [ ]:
num_tick_labels = np.linspace(start = 0, stop = 10, num = 11, dtype = int)
date_ticks = [convert_to_datetime(x) for x in num_tick_labels]

fig = ff.create_gantt(df)
fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
fig.write_html('first_figure.html', auto_open=True)
fig.show()